In [1]:
#!/usr/bin/python3

import cobra 
import pandas as pd
import re
import sys
import getopt
import os.path
import copy
import csv
import math
import cobra.flux_analysis.variability
import subprocess
import shutil, errno
import statistics
from cobra import Reaction
import cometspy as c
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams.update({'figure.max_open_warning': 0})
from importExcelModel3 import *

def initialize_models(model,tmp_id,glu, mutations=False, percentage=None):

    model2=cobra.io.read_sbml_model(model)
    modelcomets = c.model(model2)
    modelcomets.id=tmp_id
    modelcomets.change_bounds('EX_glc__D_e', -10, 0)    
    modelcomets.change_bounds('EX_fru_e', -10, 0)
    modelcomets.change_bounds('EX_malt_e', -10, 0)
    modelcomets.change_bounds('EX_lcts_e', -10, 0)
    modelcomets.change_bounds('EX_sucr_e', -5, 0)
    modelcomets.change_bounds('EX_gal_e', -10, 0)
    modelcomets.write_comets_model()
    return modelcomets
        
def model_modifications(tpha, biomass1, biomass2, model_id, gly=None, dhbz=None):
    # Single GEMs parameter modifications

    # 1.1.- Establish modifications in model  
    #You can change the bounds of a reaction using the change_bounds(reaction name, lower bound, upper bound) method
    #You can create certain conditions for your simulation
    model1 =c.model('path')
    
    """if gly is not None:
        model1.change_bounds('EX_glycol_e', gly, gly)
    if dhbz is not None:
        model1.change_bounds('EX_34dhbz_e', dhbz, 0)
    if tpha<=0:
        model1.change_bounds('ACtex',-1000,-tpha)
        model1.change_bounds('EX_ac_e', tpha, 1000)
    else:
        model1.change_bounds('ACtex', -tpha,1000)
        model1.change_bounds('EX_ac_e',-1000,tpha)"""
    model1.id=model_id[0]
    model1.write_comets_model() #write the modified model with a temporal name so the original model is not modified
    del(model1)
     # 1.2.-  Establish modifications in model 2
    model2=c.model('path')
    model2.id=model_id[1]
    model2.write_comets_model()        
    del(model2)

    return (print('Modifications of the model done!'))
    
    
def initialize_layout(filename):
    #The layout is a file with the stablished format
    layout=c.layout(filename)
    #if you need to optimize biomass, its value is changed here 
    #layout.initial_pop=[[0.0,0.0,float(biomass1),float(biomass2)]]
    return layout

def initialize_params(package, globall):
    """Function to initialize the comets' params class
    it can be initialize by submitting two files, one for the package parameters
    and one for the global ones.
    If you don't submit a file, the params class will be initialize with the values stated below
    which have been tested in this simulation"""
    
    if package and globall is not None:
        params = c.params(global_params = globall, package_params= package)
    elif package is not None:
        params = c.params(package_params=package)
    elif globall is not None:
        params = c.params(global_params=globall)
        
    else:
        params = c.params()
        params.all_params['maxCycles']=240
        params.all_params['timeStep']=0.1
        params.all_params['spaceWidth']=0.05
        params.all_params['allowCellOverlap']= True
        params.all_params['deathRate']= 0.0
        params.all_params['numRunThreads']= 8
        params.all_params['maxSpaceBiomass']= 1000
        params.all_params['defaultVmax']=20
        params.all_params['showCycleTime']=True
        params.all_params['writeTotalBiomassLog']=True
        params.all_params['writeMediaLog']=True
        params.all_params['writeFluxLog']=True
        params.all_params['useLogNameTimeStamp']=False
        params.all_params['FluxLogRate']=1
        params.all_params['MediaLogRate']=1
        params.all_params['exchangestyle']='Standard FBA'

        
    #check if param 'maxSpaceBiomass' has default value
    if (params.all_params['maxSpaceBiomass']== 0.1):
        print('The parameter "maxSpaceBiomass" is set to 0.1.\n' \
              'It may need to change if the mo used growths well.')

    return params

def make_df(strains,metabolites, comets,params,name=None):
    '''This function creates a figure and saves it to pdf format.
    It also creates the file biomass_vs_met.txt which contais the quantity
    of each strain and metabolite and has the following columns:
    time(h), strain1 ... strainX, met1 ... metX.'''
    if name==None:
        file_name='_'.join(metabolites)
    else:
        file_name=name
    df = comets.media #We get the media composition results'
    df2=comets.total_biomass #We get the biomass results
    columns=['cycle']
    for i in range(0,len(strains)):
        columns.append(strains[i])
    df2.columns=columns
    max_cycles=params.all_params['maxCycles']
    """For each metabolite a column with all zeros is added to the dataframe and each row that contains a value
     (metabolite concentration)is changed in the dataframe"""
    for d in metabolites:
        met =df[df['metabolite'] == d]

        temp=np.zeros(max_cycles+1) #Create an array with all zeros
        df2[d]=temp #We added it to the dataframe
        j=1
        while j < (max_cycles+1): #For each cycle
            if (met.cycle==j).any(): #If the row exists
                df2.loc[j-1,d] = float(met[met['cycle']==j]['conc_mmol']) #Its dataframe value is changed
            j+=1  
    
    np.savetxt(r'biomass_vs_'+file_name+'_template.txt', df2.values, fmt='%s',delimiter='\t') #The data is saved
    return df2

def make_graph(strains,metabolites, df2, GR, yield_, maxBiomass, name=None):
    if name==None:
        file_name='_'.join(metabolites)
    else:
        file_name=name
    #---Starting the figure 
    plt.ioff()
    fig, ax = plt.subplots()

    ax.set_xlabel('time (h)')
    ax.set_ylabel('biomass (g/L)')
    c=['k', 'm', 'b', 'g', 'r']
    j=0
    for i in strains:
        ax.plot(df2['cycle']*0.1, df2[i], label=i, color=c[j])
        j+=1
    ax2 = ax.twinx()
    ax2.set_ylabel('metabolite conc (mM)')
    for m in metabolites:
        ax2.plot(df2['cycle']*0.1, df2[m], label=m)

    handles, labels = ax.get_legend_handles_labels()
    handle_list, label_list = [], []

    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)
    handles, labels = ax2.get_legend_handles_labels()
    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)
    plt.legend(handle_list, label_list,bbox_to_anchor=(0.6,0.2))
    
    data = [[round(GR,2), round(yield_,2), round(maxBiomass,2)]]
    table = plt.table(cellText=data, colLabels=['Growth Rate (µmax)', 'Biomass Yield', 'Final Biomass (g/L)'], loc='bottom', 
                  cellLoc='center', colColours=['#66A4A9', '#D594D6', '#94D6AD'], bbox=[0.0,-0.5,1,.28])
    #saving the figure to a pdf
    plt.subplots_adjust(left=0.2, bottom=0.2)
    #plt.show()
    plt.tight_layout()
    plt.savefig('biomass_vs_'+file_name+'_template_plot.pdf')
    return df2

def end_simulation_cycle(gly, met, strains, df2, met2=None, met3=None, met4=None, met5=None):
    """function that stablishes the endCyle after a certain stop condition
    If the initial biomass of a microorganism needs to be used, you can access it with
    float(df2.at[0,'Ecoli1']), where df2 is the dataframe created in make_graph() function.
    The following is a example of a stop condition in which the cycle is stopped when either:
    glucose and acetate are exhausted or when glucose is exhausted and the microorganisms
    can't use acetate to grow.
    The stop condition should be suitable for your simulation.
    """
    iniBiomass=float(df2.at[0,strains[0]])
    totTpha=float(df2.at[0,met])
    #totmet2=float(df2.at[0,met2])
    #totmet3=float(df2.at[0,met3])
    #totmet4=float(df2.at[0,met4])
    #totmet5=float(df2.at[0,met5])
    endTphaCycle=0
    for row in df2.iterrows():
        endCycle=int(row[1][0])
        tphaConc=float(row[1][2])
        if tphaConc==0: #This is an example that stops the simulation if concentrations of
            endTphaCycle=endCycle
            break
    return iniBiomass, endTphaCycle, totTpha

def biomass_yield(df2,endCycle, strains):
    # Function that compute final biomass as the maximum biomass of each strain.
    
    finalBiomass={}
    for i in range(1, len(strains)+1):
        finalBiomass['finalBiomass'+str(i)]=0
        count=0
        for row in df2.iterrows():
            if float(row[1][i])>finalBiomass['finalBiomass'+str(i)]:
                finalBiomass['finalBiomass'+str(i)]=float(row[1][i])
            if count > endCycle:
                break
            count+=1
        total_finalBiomass=sum(finalBiomass.values())
    return total_finalBiomass
    

def compute_fitness(MaximumYield, finalBiomass, iniBiomass, fitFunc, fitTime, comets,layout,model):
    """Establising a fitness function is one of the most critical steps in FLYCOP. There needs to be carefully
    thinked. Here are some examples that can be totally rewritable"""
    if(fitFunc=='Yield'):
        fitness=(biomassYieldNew/MaximumYield) # Normalizing Yield
    elif(fitFunc=='MaxYield_MinTime'):
        fitness=0.5*(biomassYieldNew/MaximumYield)+0.5*fitTime #Normalizing yield
    elif(fitFunc=='YieldNewScattered'): # (biomass^4)*10: To spread values from ~0.45-0.55 values to 0.5 to 1
        fitness=(biomassYieldNew**4)*10
    elif(fitFunc=='MaxYieldNewScattered_MinTime'):
        fitness=0.5*((biomassYieldNew**4)*10)+0.5*fitTime
    elif(fitFunc=='Biomass'):
        fitness=float(finalBiomass-iniBiomass)
    elif(fitFunc=='MaxBiomass_MinTime'):
        fitness=0.5*(float(finalBiomass-iniBiomass))+0.5*fitTime
    elif((fitFunc=='GR')or(fitFunc=='MaxGR_MinTime')):
        flux = comets.fluxes
        gr=[]
        #model_tmp = c.model("model_1_tmp.cmd")
        objective = model.objective
        gr.append(flux.iat[47-1,int(objective+3)])
        fitGR=sum(gr)/len(gr)
    if fitFunc == 'GR':
        fitness=fitGR
    elif(fitFunc=='MaxGR_MinTime'):
        fitness=0.5*fitGR+0.5*fitTime
    #if(float(finalBiomass-iniBiomass) > 1.03):  # Given that with both strains with WT, total biomass=1.028
    #    fitness=0
    return fitness

def calculate_uptake(comets,layout, met):
    uptakeMet=[]
    flux=comets.fluxes
        #Compute metabolite uptake
    model=layout.models[0]
    df3 = model.reactions
    for me in met:
        j=0
        numRxnMet=int(df3.loc[df3['REACTION_NAMES']=='EX_'+str(me)]['ID'])
        uptakeMet.append(float(flux.iat[j+2,numRxnMet+3]))
        j+=1
    return uptakeMet


def Flycop(biomass1,filename, fitFunc='GR', dirPlot='', repeat=2, params_package=None, params_global=None):
    """Primarily function of the simulation
    You need to add to it the parameters to adjust.
    You need to fill these variables below(layout, model_id, model, wgMet, metabolites, strains) with your information
    layout= The layout for the comets simulation
    model_id = the names of the models in your layout
    model= the name of the model yours modifications are from
    wgMet=molecular weight of the metabolite to compute fitness. Must be divided by 1000.
    metabolites= list with the names of the metabolites to study
    strains = list with the names of the strains you're working with
    simulationID = will be part of the filename for the results 
    metUptake = the metabolite you want to calculate the uptake of"""
    model_file='Final_MOA199.xml'
    layout_file='DEMO1.txt'
    model_id='model_1_tmp' #This are the model names that have to be in the layout
    wgMet=[0.180156] #molecular weigh of the metabolite that you want to calculate the uptake of
    metabolites=['sucr_e', 'fru_e','glc__D_e','nh4_e', 'o2_e'] #Metabolite names as they are written in the layout
    strains=['P.megaterium'] #Name of the strains - going to apper in the graph
    metUptake = ['o2_e', 'co2_e']
    
    if not(os.path.exists('IndividualRunsResults')):
        os.makedirs('IndividualRunsResults')
    
    totfitness=0
    sumTotBiomass=0
    sumTotYield=0
    fitnessList=[]
    print('Initilazing model...')
    modelcomets=initialize_models(model_file,model_id,filename)
    print('Initialazing layout...')
    layout= initialize_layout(layout_file)
    print('Done!')
    print('Initialazing params...')

    params =initialize_params(params_package, params_global)
   
    #establish comets class
    comets = c.comets(layout, params)
    """You may need to adjust the comets.set_classpath object if
    your programs aren't in the default comets location.
    Changing using the comets method set_classpath('name of the program', 'path')"""
    #comets.set_classpath('lang3', '/home/hugo/comets/lib/commons-lang3-3.9/commons-lang3-3.9.jar')
   

    # To repeat X times, due to random behaviour in COMETS:
    for i in range(repeat):

        comets.run(delete_files=False)
        
        #obtaining the results and writing them to csv
        comets.total_biomass.to_csv('Total_biomass_log_template.txt',sep='\t',index=False)
        comets.fluxes.to_csv('Flux_log_template.txt',sep='\t', index=False)
        comets.media.to_csv('Media_log_template.txt',sep='\t', index=False)        
        
        #make the graphic
        df2=make_df(strains,metabolites, comets, params,filename)
        # 7.- Compute fitness (measure to optimize):
        print('computing fitness...')
        
        # 7.1.- Determine endCycle: when glucose and acetate are exhausted
        iniBiomass, endCycle, totMetSucr = end_simulation_cycle(filename, metabolites[0], strains,df2)
        
        # 7.2.- Compute first element fitness: maximize biomass yield
        finalBiomass =biomass_yield(df2,endCycle, strains)
        if wgMet is not None:
            #totmet=((totMetSucr*wgMet[0])+(totMetGlc*wgMet[1])+(totMetmal*wgMet[2])+(totMetFru*wgMet[3])+(totMetLac*wgMet[4]))
            biomassYieldNew=float(finalBiomass-iniBiomass)/(totMetSucr*wgMet[0]) # molecular weigth of met per mmol
        # For normalizing yield
        MaximumYield=1
        
        # 7.3.- Compute second element fitnes: minimize time        
        fitTime=1-(float(endCycle)/float(params.all_params['maxCycles']))
        
        # 7.4.- Compute joint fitness, as a 50% each element.
        fitness= compute_fitness(MaximumYield, finalBiomass, iniBiomass, fitFunc, fitTime, comets, layout,modelcomets)
        # 7.5.- Calculate the uptake of metabolite
        uptakeMet =calculate_uptake(comets,layout, metUptake)
        make_graph(strains,metabolites, df2, fitness, biomassYieldNew, finalBiomass, filename)
            
            
        print(" Total biomass: "+str(round(finalBiomass,6))+" in cycle "+str(endCycle)+". Biomass yield="+str(round(biomassYieldNew,6)))
   
        totfitness=totfitness+fitness
        fitnessList.append(fitness)
        sumTotBiomass=sumTotBiomass+finalBiomass
        sumTotYield=sumTotYield+biomassYieldNew
            
        # Copy individual solution
        #x= '_'.join(metabolites)
        x=filename
        y='_'.join(metUptake)
        file='IndividualRunsResults/'+'biomass_vs_'+x+'_run'+str(i)+'_'+str(fitness)+'_'+str(endCycle)+'.pdf'        
        shutil.move('biomass_vs_'+x+'_template.txt',file)
        if(dirPlot != ''):
            file2=dirPlot+'biomass_vs_'+x+'_'+y+'_run'+str(i)+'_'+str(fitness)+'_'+str(endCycle)+'.pdf'
            shutil.copy(file,file2)
        file='IndividualRunsResults/'+'total_biomass_log_run'+str(i)+'.txt'
        shutil.move('Total_biomass_log_template.txt',file)
        file='IndividualRunsResults/'+'media_log_run'+str(i)+'.txt'
        shutil.move('Media_log_template.txt',file)
        file='IndividualRunsResults/'+'flux_log_run'+str(i)+'.txt'
        shutil.move('Flux_log_template.txt',file) 
            
    avgfitness=totfitness/repeat
    sdfitness=statistics.stdev(fitnessList)
    avgBiomass=sumTotBiomass/repeat
    avgYield=sumTotYield/repeat
    
    print("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle\tUptakeMet")
                    
    print(fitFunc+"\t"+str(filename)+str(biomass1)+','+y+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle)+"\t"+str(uptakeMet))
    with open(dirPlot+"configurationsResults"+fitFunc+".txt", "a") as myfile:
        myfile.write("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle\n")
        myfile.write(fitFunc+"\t"+str(filename)+','+str(biomass1)+','+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle)+"\t"+str(uptakeMet)+"\n")
  
    print("Avg.fitness(sd):\t"+str(avgfitness)+"\t"+str(sdfitness)+"\n")
    if(sdfitness>0.1):
        avgfitness=0.0
    
    #tmp models needs to be deleted
    """for i in model_id:
        try:
            os.remove(str(i)+'.cmd')
        except FileNotFoundError:
            pass"""

    return comets

In [2]:
comets=Flycop(0.01, 'DEMO1')
comets.run_output

Initilazing model...


/home/hugo/anaconda3/envs/cobranp22/lib/python3.10/site-packages/cometspy/model.py:973: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  smat.to_csv(f, mode='a', line_terminator = '\n', header=False, index=False)
/home/hugo/anaconda3/envs/cobranp22/lib/python3.10/site-packages/cometspy/model.py:979: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  bnd.to_csv(f, mode='a', line_terminator = '\n', header=False, index=False)
/home/hugo/anaconda3/envs/cobranp22/lib/python3.10/site-packages/cometspy/model.py:987: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  met_n.to_csv(f, mode='a', line_terminator = '\n', header=False, index=False)
/home/hugo/anaconda3/envs/cobranp22/lib/python3.10/site-packages/cometspy/model.py:991: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  rxn_n.to_csv(f, mode='a', line_terminator = '\n'

Initialazing layout...
Done!
Initialazing params...

Running COMETS simulation ...


/home/hugo/anaconda3/envs/cobranp22/lib/python3.10/site-packages/cometspy/model.py:973: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  smat.to_csv(f, mode='a', line_terminator = '\n', header=False, index=False)
/home/hugo/anaconda3/envs/cobranp22/lib/python3.10/site-packages/cometspy/model.py:979: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  bnd.to_csv(f, mode='a', line_terminator = '\n', header=False, index=False)
/home/hugo/anaconda3/envs/cobranp22/lib/python3.10/site-packages/cometspy/model.py:987: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  met_n.to_csv(f, mode='a', line_terminator = '\n', header=False, index=False)
/home/hugo/anaconda3/envs/cobranp22/lib/python3.10/site-packages/cometspy/model.py:991: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  rxn_n.to_csv(f, mode='a', line_terminator = '\n'

Error: COMETS simulation did not complete

     examine comets.run_output for the full java trace

     if we detect a common reason, it will be stated in the RuntimeError at the bottom


RuntimeError: COMETS simulation did not complete:
 undetected reason. examine comets.run_output for JAVA trace